# Sector rotation 

In [ ]:
%load_ext autoreload
%autoreload 2
import sector_rot
import pandas as pd
from pathlib import Path
import os
import mlflow
import subprocess
import time

/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [16]:
def kill_process_on_port(port):
    try:
        result = subprocess.check_output(f"lsof -ti tcp:{port}", shell=True, text=True)
        pids = result.strip().split('\n')
        for pid in pids:
            if pid:
                print(f"Killing process {pid} on port {port}")
                os.system(f"kill -9 {pid}")
    except subprocess.CalledProcessError:
        print(f"No process found on port {port}")

def start_mlflow_ui(port=5000):
    kill_process_on_port(port)
    print(f"Starting MLflow UI on port {port} ...")
    subprocess.Popen(
        ["mlflow", "ui", "--host", "127.0.0.1", "--port", str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    time.sleep(3)
    print(f"MLflow UI running at http://127.0.0.1:{port}")

start_mlflow_ui()

Killing process 6976 on port 5000
Killing process 6988 on port 5000
Killing process 6989 on port 5000
Killing process 6991 on port 5000
Killing process 6992 on port 5000
Killing process 6993 on port 5000
Starting MLflow UI on port 5000 ...
MLflow UI running at http://127.0.0.1:5000


In Simonian et al. (2019) the Fama–French–Carhart (FFC) factor **realisations for the current month *t*** are fed straight into a Random-Forest model as predictor variables (“features”) to generate a point estimate of each sector’s excess return⁠—what the paper calls the “RF-predicted return”﻿.

That RF-predicted return is then used **as a trading signal for the *next* month ( *t + 1*)** inside the association-rule-learning (ARL) overlay that powers the sector-rotation strategy:

> “the signals are the RF-predicted return of a sector … and the ratio of volatilities … If … the RF-predicted return for **next month** is greater than a designated threshold value, then we will own the sector for the month”﻿.

So the workflow is:

1. **Month *t***

   * Observe the four FFC factor returns (MKT, SMB, HML, MOM).
   * Feed them into the trained RF to obtain a *contemporaneous* predicted sector return.

2. **Month *t + 1***

   * Treat that predicted value (together with a volatility-ratio signal) as an input to ARL rules that decide whether the sector is held during month *t + 1*.
   * Evaluate the realised return over month *t + 1*.

### What the model **does not** do

* It never forecasts the factor returns themselves for *t + 1*; it simply uses the observed factor values at *t*.
* The risk-decomposition (pseudo-beta) exercise appears later in the article and is presented only as an interpretability device—translating RF feature importances into something that looks like traditional betas﻿. Those pseudo-betas are **not** fed back into the predictive model or the trading rules.

**Bottom line:** the author uses the month-*t* Fama–French factor returns directly to produce a model-based prediction of sector returns, and that prediction becomes one of the signals for trading one month ahead; factor-risk decompositions are used solely for ex-post interpretation, not for forecasting.


In [2]:
%cd ..

/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis


In [17]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")


In [3]:
data_dir = Path.cwd()/'data'

df_dict = {
    file.stem.replace("sector_","") : pd.read_parquet(file)
    for file in data_dir.glob("sector_*.parquet")
}

/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/pandas/co

In [4]:
df_dict['10'].tail(50)

,vol,ret,shrout,prc,askhi,bidlo,put_volume,call_volume,put_call_ratio,vix_close,...,enhanced_baker,news_sent,mktcap,turn_sd,sect_mktcap,mvel1,dolvol,daily_illq,excess_ret,excess_mkt_ret
index,,,,,,,,,,,,,,,,,,,,,
2018-10-17,7.090816e+06,-0.006811,1.790268e+06,82.103614,82.844743,81.218036,5257.450597,9271.779058,0.738169,17.400000,...,1.677,0.03,1.475542e+08,3.072879,1.469875e+08,18.809706,5.821816e+08,1.169872e-05,-0.006891,-0.00088
2018-10-18,9.135598e+06,-0.004694,1.799754e+06,81.919688,82.662846,81.060460,7247.380047,14073.035953,0.803405,20.059999,...,1.677,0.02,1.486415e+08,3.072879,1.474353e+08,18.817048,7.483854e+08,6.271510e-06,-0.004774,-0.01548
2018-10-19,8.483855e+06,-0.007025,1.813272e+06,81.210296,82.805008,80.770177,9356.581814,13284.470162,0.809722,19.889999,...,1.677,-0.01,1.497707e+08,3.072879,1.472564e+08,18.824616,6.889764e+08,1.019680e-05,-0.007105,-0.00258
2018-10-22,6.825382e+06,-0.011078,1.813387e+06,80.534156,81.618499,79.809177,5923.296388,10345.804992,0.831708,19.639999,...,1.677,0.00,1.484687e+08,3.072879,1.460396e+08,18.815885,5.496764e+08,2.015399e-05,-0.011158,-0.00388
2018-10-23,9.712357e+06,-0.026617,1.821805e+06,78.276509,79.388233,76.988453,10144.860877,17747.840518,0.586361,20.709999,...,1.677,-0.01,1.456699e+08,3.072879,1.426045e+08,18.796854,7.602494e+08,3.501068e-05,-0.026697,-0.00628
2018-10-24,1.029028e+07,-0.037594,1.834264e+06,75.423177,78.905061,75.282592,9503.541755,19322.373138,0.623939,25.230000,...,1.677,0.00,1.422711e+08,3.072879,1.383460e+08,18.773245,7.761257e+08,4.843839e-05,-0.037674,-0.03338
2018-10-25,8.117874e+06,0.011599,1.832241e+06,76.491694,77.563637,75.648372,8404.785245,11966.378354,1.098704,24.219999,...,1.677,-0.01,1.437202e+08,3.072879,1.401512e+08,18.783379,6.209500e+08,1.868014e-05,0.011519,0.01922
2018-10-26,1.069092e+07,-0.007505,1.832931e+06,76.014522,77.041966,74.575708,8527.553713,10198.959978,0.853735,24.160000,...,1.677,-0.01,1.428546e+08,3.072879,1.393294e+08,18.777338,8.126652e+08,9.235131e-06,-0.007585,-0.01658
2018-10-29,9.453560e+06,-0.018507,1.844366e+06,74.526563,76.830017,73.560570,8728.831942,8719.024538,0.958609,24.700001,...,1.677,0.02,1.417949e+08,3.072879,1.374543e+08,18.769892,7.045414e+08,2.626841e-05,-0.018587,-0.00778


# Playground 

In [6]:
!pwd

/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis


/opt/anaconda3/envs/mscthesis/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=5073) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [5]:
%cd vsc/erasmus/msc_thesis/

[Errno 2] No such file or directory: 'vsc/erasmus/msc_thesis/'
/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis


In [9]:
test_df_2018 = df_dict['20'].loc[df_dict['20'].index.year == 2018]

In [10]:
test_df_2018

,vol,ret,shrout,prc,askhi,bidlo,put_volume,call_volume,put_call_ratio,vix_close,...,enhanced_baker,news_sent,mktcap,turn_sd,sect_mktcap,mvel1,dolvol,daily_illq,excess_ret,excess_mkt_ret
index,,,,,,,,,,,,,,,,,,,,,
2018-01-02,7.253433e+06,0.007141,964807.145901,147.795301,148.607363,146.341177,7655.451382,14661.520806,1.223743,9.770000,...,1.896,0.26,7.363997e+07,2.646192,1.425940e+08,18.114698,1.072023e+09,0.000007,0.007081,0.00844
2018-01-03,7.819286e+06,0.005477,963128.522592,148.831343,149.219216,147.275353,9806.977659,21366.047542,1.027184,9.150000,...,1.896,0.28,7.377911e+07,2.646192,1.433437e+08,18.116586,1.163755e+09,0.000005,0.005417,0.00584
2018-01-04,7.954572e+06,0.007758,969619.343062,149.781473,150.525103,148.403776,9298.970420,17586.649220,1.464557,9.220000,...,1.896,0.25,7.439266e+07,2.646192,1.452310e+08,18.124868,1.191447e+09,0.000007,0.007698,0.00414
2018-01-05,7.594680e+06,0.007178,966796.240046,151.870665,152.625760,149.765135,11358.584001,21199.396245,0.904224,9.220000,...,1.896,0.25,7.557459e+07,2.646192,1.468280e+08,18.140631,1.153409e+09,0.000006,0.007118,0.00654
2018-01-08,7.760005e+06,0.003377,958062.361779,152.661489,153.413298,151.292675,13645.154134,15826.973276,1.478964,9.520000,...,1.896,0.28,7.565293e+07,2.646192,1.462592e+08,18.141667,1.184654e+09,0.000003,0.003317,0.00184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,1.159437e+07,-0.019646,702305.780691,131.081914,135.385748,130.621275,23346.189328,9808.478934,1.941784,30.110001,...,2.409,-0.04,5.949491e+07,2.646192,9.205959e+07,17.901401,1.519812e+09,0.000013,-0.019756,-0.02181
2018-12-24,4.246715e+06,-0.031127,701384.237663,126.805456,130.637657,126.559516,5492.626611,5429.304035,2.125264,36.070000,...,2.409,-0.04,5.741706e+07,2.646192,8.893935e+07,17.865852,5.385067e+08,0.000058,-0.031237,-0.02561
2018-12-26,7.221482e+06,0.047993,707331.184670,132.927957,133.038597,125.796325,6986.219092,12610.822473,1.265914,30.410000,...,2.409,-0.08,6.063741e+07,2.646192,9.402409e+07,17.920423,9.599369e+08,0.000050,0.047883,0.05049


In [11]:
test_result_dict= sector_rot.rolling_pred(
    945551225694133017,
    "0def5ed344304dc5b8e85d4faebeb29d",
    df = test_df_2018,
    lookback_time=50,
    vol_threshold = 1.0,
    pred_thresh = 0.0,
    excess_ret_pred_threshold = 0.0,
    sr = 5,
    lr = 21,
    experiment_name= 'pre_test').fit()
    

MLRuns path: py/mlruns/945551225694133017
Meta path: py/mlruns/945551225694133017/meta.yaml
Meta path exists: True
MLRuns path: py/mlruns/945551225694133017
Meta path: py/mlruns/945551225694133017/meta.yaml
Meta path exists: True

=== Debugging _extract_features ===
Looking for factors file at: py/mlruns/945551225694133017/0def5ed344304dc5b8e85d4faebeb29d/params/factors
Features loaded: ['excess_mkt_ret', 'smb', 'hml', 'umd', 'cma']
Experiment already exists with ID: 490636618977650074
Dumping models

=== Debugging _dump_model ===

=== Debugging _extract_model_pkl ===
Checking if meta.yaml exists: True
Meta content: {'artifact_location': 'mlflow-artifacts:/945551225694133017', 'creation_time': 1748356255792, 'experiment_id': '945551225694133017', 'last_update_time': 1748356255792, 'lifecycle_stage': 'active', 'name': 'baseline_ols'}
meta.yaml name extract: baseline_ols
OLS pattern: py/mlartifacts/945551225694133017/0def5ed344304dc5b8e85d4faebeb29d/artifacts/ols_model/*.statsmodels
OLS 

/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis/py/sector_rot.py:226: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.ols_prediction_series.iloc[t] = trained_ols.predict(X_test_const)[0]
/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis/py/sector_rot.py:226: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.ols_prediction_series.iloc[t] = trained_ols.predict(X_test_const)[0]
/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis/py/sector_rot.py:226: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with

Signal set generated: index
2018-10-17    0
2018-10-18    0
2018-10-19    0
2018-10-22    0
2018-10-23    0
2018-10-24    0
2018-10-25    0
2018-10-26    0
2018-10-29    0
2018-10-30    0
2018-10-31    0
2018-11-01    0
2018-11-02    0
2018-11-05    0
2018-11-06    0
2018-11-07    0
2018-11-08    0
2018-11-09    0
2018-11-12    0
2018-11-13    0
2018-11-14    0
2018-11-15    0
2018-11-16    0
2018-11-19    0
2018-11-20    0
2018-11-21    0
2018-11-23    0
2018-11-26    0
2018-11-27    0
2018-11-28    0
2018-11-29    0
2018-11-30    0
2018-12-03    0
2018-12-04    0
2018-12-06    0
2018-12-07    0
2018-12-10    0
2018-12-11    0
2018-12-12    0
2018-12-13    0
2018-12-14    0
2018-12-17    0
2018-12-18    0
2018-12-19    0
2018-12-20    0
2018-12-21    0
2018-12-24    0
2018-12-26    0
2018-12-27    0
2018-12-28    0
Name: signal, dtype: int64
Rule diagnostics df generated:              support  confidence      lift
idx                                       
2018-10-16  0.137255        

In [25]:
test_result_dict['rf_signal_set'].tail(100)

index
2018-08-07    1
2018-08-08    1
2018-08-09    1
2018-08-10    0
2018-08-13    0
             ..
2018-12-21    0
2018-12-24    0
2018-12-26    0
2018-12-27    0
2018-12-28    0
Name: signal, Length: 100, dtype: int64

In [10]:
ols_pred

index
1998-01-02   NaN
1998-01-05   NaN
1998-01-06   NaN
1998-01-07   NaN
1998-01-08   NaN
              ..
1999-12-20   NaN
1999-12-21   NaN
1999-12-22   NaN
1999-12-23   NaN
1999-12-27   NaN
Length: 500, dtype: float64

In [142]:
rf_pred

index
2018-10-17         NaN
2018-10-18         NaN
2018-10-19         NaN
2018-10-22         NaN
2018-10-23         NaN
2018-10-24         NaN
2018-10-25         NaN
2018-10-26         NaN
2018-10-29         NaN
2018-10-30         NaN
2018-10-31         NaN
2018-11-01         NaN
2018-11-02         NaN
2018-11-05         NaN
2018-11-06         NaN
2018-11-07         NaN
2018-11-08         NaN
2018-11-09         NaN
2018-11-12         NaN
2018-11-13         NaN
2018-11-14         NaN
2018-11-15         NaN
2018-11-16         NaN
2018-11-19         NaN
2018-11-20         NaN
2018-11-21         NaN
2018-11-23         NaN
2018-11-26         NaN
2018-11-27         NaN
2018-11-28         NaN
2018-11-29         NaN
2018-11-30   -0.006386
2018-12-03    0.006566
2018-12-04    0.008382
2018-12-06   -0.002721
2018-12-07   -0.003864
2018-12-10   -0.013041
2018-12-11   -0.001130
2018-12-12   -0.006713
2018-12-13    0.005548
2018-12-14   -0.007802
2018-12-17   -0.006931
2018-12-18   -0.010017
2018-

In [148]:
# Create matched dataframe with rf predictions and excess returns
matched_df = pd.DataFrame({
    'excess_ret': test_df['excess_ret'],
    'preds': rf_pred
}).dropna()


In [149]:
matched_df

,excess_ret,preds
index,,
2018-11-30,-0.002323,-0.006386
2018-12-03,0.022875,0.006566
2018-12-04,-0.028615,0.008382
2018-12-06,-0.017293,-0.002721
2018-12-07,-0.006089,-0.003864
2018-12-10,-0.016160,-0.013041
2018-12-11,0.000425,-0.001130
2018-12-12,0.003378,-0.006713
2018-12-13,0.004085,0.005548


In [9]:
sector_rot.rolling_pred(
    208039388113350502,
    "f5b7855c3eae48f18c61879afbc7e95e",
    df = df_dict['25'],
    lookback_time= 365,
    vol_threshold = 1.0,
    pred_thresh = 0.0,
    excess_ret_pred_threshold = 0.0,
    sr = 21,
    lr = 126,
    experiment_name= 'rf_test_arl'
)._extract_sector_factors()

MLRuns path: py/mlruns/208039388113350502
Meta path: py/mlruns/208039388113350502/meta.yaml
Meta path exists: True
Extracting sector number for run f5b7855c3eae48f18c61879afbc7e95e
Looking for sectors file at: py/mlruns/208039388113350502/f5b7855c3eae48f18c61879afbc7e95e/params/sector
Extracting factors
Sectors loaded: 20_rf
Factors loaded: ['excess_mkt_ret', 'smb', 'hml', 'umd']


('20_rf', "['excess_mkt_ret', 'smb', 'hml', 'umd']")

In [16]:
first_rf_pass = sector_rot.rolling_pred(
    945551225694133017,
    "0def5ed344304dc5b8e85d4faebeb29d",
    df = df_dict['20'],
    lookback_time= 365,
    vol_threshold = 1.0,
    pred_thresh = 0.0,
    excess_ret_pred_threshold = 0.0,
    sr = 5,
    lr = 21,
    experiment_name= 'pre_test').fit()


MLRuns path: py/mlruns/945551225694133017
Meta path: py/mlruns/945551225694133017/meta.yaml
Meta path exists: True
MLRuns path: py/mlruns/945551225694133017
Meta path: py/mlruns/945551225694133017/meta.yaml
Meta path exists: True

=== Debugging _extract_features ===
Looking for factors file at: py/mlruns/945551225694133017/0def5ed344304dc5b8e85d4faebeb29d/params/factors
Features loaded: ['excess_mkt_ret', 'smb', 'hml', 'umd', 'cma']
Experiment already exists with ID: 490636618977650074
Dumping models

=== Debugging _dump_model ===

=== Debugging _extract_model_pkl ===
Checking if meta.yaml exists: True
Meta content: {'artifact_location': 'mlflow-artifacts:/945551225694133017', 'creation_time': 1748356255792, 'experiment_id': '945551225694133017', 'last_update_time': 1748356255792, 'lifecycle_stage': 'active', 'name': 'baseline_ols'}
meta.yaml name extract: baseline_ols
OLS pattern: py/mlartifacts/945551225694133017/0def5ed344304dc5b8e85d4faebeb29d/artifacts/ols_model/*.statsmodels
OLS 

/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis/py/sector_rot.py:226: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.ols_prediction_series.iloc[t] = trained_ols.predict(X_test_const)[0]
/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis/py/sector_rot.py:226: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.ols_prediction_series.iloc[t] = trained_ols.predict(X_test_const)[0]
/Users/minhquangngo/Documents/vsc/erasmus/msc_thesis/py/sector_rot.py:226: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with

OLS predictions generated: index
1998-01-02   NaN
1998-01-05   NaN
1998-01-06   NaN
1998-01-07   NaN
1998-01-08   NaN
dtype: float64
Generating apriori df....
Apriori df generated:             low_vr  high_pred  ret_up
index                                
1998-01-02   False      False    True
1998-01-05   False      False    True
1998-01-06   False      False   False
1998-01-07   False      False    True
1998-01-08   False      False   False
Generating arl set....
Signal set generated: index
2018-10-17    0
2018-10-18    0
2018-10-19    0
2018-10-22    0
2018-10-23    0
2018-10-24    0
2018-10-25    0
2018-10-26    0
2018-10-29    0
2018-10-30    0
2018-10-31    0
2018-11-01    0
2018-11-02    0
2018-11-05    0
2018-11-06    1
2018-11-07    1
2018-11-08    1
2018-11-09    0
2018-11-12    0
2018-11-13    0
2018-11-14    0
2018-11-15    0
2018-11-16    1
2018-11-19    1
2018-11-20    0
2018-11-21    0
2018-11-23    0
2018-11-26    0
2018-11-27    0
2018-11-28    1
2018-11-29    1
2018-1

# Extracting signals

In [23]:
experiments = sector_rot.all_runs(experiment_number=None).get_experiments()

/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/IPython/extensions/deduperreload/deduperreload.py:290: DeprecationWarning: ast.Ellipsis is deprecated and will be removed in Python 3.14; use ast.Constant instead
  elif not isinstance(ast_elt, (ast.Ellipsis, ast.Pass)):
/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/IPython/extensions/deduperreload/deduperreload.py:290: DeprecationWarning: ast.Ellipsis is deprecated and will be removed in Python 3.14; use ast.Constant instead
  elif not isinstance(ast_elt, (ast.Ellipsis, ast.Pass)):
/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/IPython/extensions/deduperreload/deduperreload.py:290: DeprecationWarning: ast.Ellipsis is deprecated and will be removed in Python 3.14; use ast.Constant instead
  elif not isinstance(ast_elt, (ast.Ellipsis, ast.Pass)):
/opt/anaconda3/envs/mscthesis/lib/python3.12/site-packages/IPython/extensions/deduperreload/deduperreload.py:290: DeprecationWarning: ast.Ellipsis is deprecated

In [24]:
experiments

['254201543281791601',
 '477147033551921912',
 '208039388113350502',
 '945551225694133017',
 '717450669580849996']

In [ ]:
rf_basepath_rf =sector_rot.all_runs(208039388113350502).get_run_folders()
for i in rf_basepath_rf:
    for turn, sector in i.items():
        print(f"Training rf baseline ARL for run {turn} of sector {sector}")
        sector_numb = ''.join(filter(str.isdigit, sector))
        arl_run = sector_rot.rolling_pred(
    208039388113350502,
    str(turn),
    df = df_dict[str(sector_numb)],
    lookback_time= 365,
    vol_threshold = 1.0,
    pred_thresh = 0.0,
    excess_ret_pred_threshold = 0.0,
    sr = 5,
    lr = 21,
    experiment_name= 'rf_base_arl').fit()


Training ARL for run f5b7855c3eae48f18c61879afbc7e95e of sector 20_rf
MLRuns path: py/mlruns/208039388113350502
Meta path: py/mlruns/208039388113350502/meta.yaml
Meta path exists: True
MLRuns path: py/mlruns/208039388113350502
Meta path: py/mlruns/208039388113350502/meta.yaml
Meta path exists: True

=== Debugging _extract_features ===
Looking for factors file at: py/mlruns/208039388113350502/f5b7855c3eae48f18c61879afbc7e95e/params/factors
Features loaded: ['excess_mkt_ret', 'smb', 'hml', 'umd']
Experiment already exists with ID: 317094863492494700
Dumping models

=== Debugging _dump_model ===

=== Debugging _extract_model_pkl ===
Checking if meta.yaml exists: True
Meta content: {'artifact_location': 'mlflow-artifacts:/208039388113350502', 'creation_time': 1748272532371, 'experiment_id': '208039388113350502', 'last_update_time': 1748272532371, 'lifecycle_stage': 'active', 'name': 'rf'}
meta.yaml name extract: rf
RF path: py/mlartifacts/208039388113350502/f5b7855c3eae48f18c61879afbc7e95e

KeyboardInterrupt: 

In [ ]:
enhanced_basepath_rf =sector_rot.all_runs(254201543281791601).get_run_folders()
for i in enhanced_basepath_rf:
    for turn, sector in i.items():
        print(f"Training ARL for run {turn} of sector {sector}")
        sector_numb = ''.join(filter(str.isdigit, sector))
        arl_run = sector_rot.rolling_pred(
    254201543281791601,
    str(turn),
    df = df_dict[str(sector_numb)],
    lookback_time= 365,
    vol_threshold = 1.0,
    pred_thresh = 0.0,
    excess_ret_pred_threshold = 0.0,
    sr = 5,
    lr = 21,
    experiment_name= 'rf_enhanced_arl').fit()


In [ ]:
enhanced_path_rf =sector_rot.all_runs(254201543281791601).get_run_folders()
for i in enhanced_path_rf:
    for turn, sector in i.items():
        print(f"Training ARL for run {turn} of sector {sector}")
        sector_numb = ''.join(filter(str.isdigit, sector))
        arl_run = sector_rot.rolling_pred(
    254201543281791601,
    str(turn),
    df = df_dict[str(sector_numb)],
    lookback_time= 365,
    vol_threshold = 1.0,
    pred_thresh = 0.0,
    excess_ret_pred_threshold = 0.0,
    sr = 5,
    lr = 21,
    experiment_name= 'rf_enhanced_arl').fit()


In [ ]:
base_path_ols =sector_rot.all_runs(945551225694133017).get_run_folders()
for i in base_path_ols:
    for turn, sector in i.items():
        print(f"Training ARL for run {turn} of sector {sector}")
        sector_numb = ''.join(filter(str.isdigit, sector))
        arl_run = sector_rot.rolling_pred(
    254201543281791601,
    str(turn),
    df = df_dict[str(sector_numb)],
    lookback_time= 365,
    vol_threshold = 1.0,
    pred_thresh = 0.0,
    excess_ret_pred_threshold = 0.0,
    sr = 5,
    lr = 21,
    experiment_name= 'ols_base_arl').fit()


In [ ]:
enhanced_path_ols =sector_rot.all_runs(717450669580849996).get_run_folders()
for i in enhanced_path_ols:
    for turn, sector in i.items():
        print(f"Training ARL for run {turn} of sector {sector}")
        sector_numb = ''.join(filter(str.isdigit, sector))
        arl_run = sector_rot.rolling_pred(
    717450669580849996,
    str(turn),
    df = df_dict[str(sector_numb)],
    lookback_time= 365,
    vol_threshold = 1.0,
    pred_thresh = 0.0,
    excess_ret_pred_threshold = 0.0,
    sr = 5,
    lr = 21,
    experiment_name= 'ols_enhanced_arl').fit()
